### Python Libraries

In [2]:
import scipy.io
import scipy.linalg as sc_linalg
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn import *
from sklearn.model_selection import KFold
from sklearn.utils import resample
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import cross_val_score
from math import sqrt

import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


### Creating Scenarios and Data preprocessing

In [3]:
df_male = pd.read_csv('MALE.csv', sep=',')
df_female = pd.read_csv('FEMALE.csv', sep=',')
df_mixed = pd.read_csv('MIXED.csv', sep=',')

# Removing VR Band of student 0
df_mixed = df_mixed[df_mixed['VR Band of Student'] != 0]

def one_hot_encoding_transformation(df):
    df_year = pd.get_dummies(df['Year'], prefix='Year')
    df_vr_band_student = pd.get_dummies(df['VR Band of Student'], prefix='VR Band of Student')
    df_ethnic_group = pd.get_dummies(df['Ethnic group of student'], prefix='Ethnic group of student')
    df_school_denomination = pd.get_dummies(df['School denomination'], prefix='School denomination')
    df_final = pd.concat([df_year, df['FSM'], df['VR1 Band'],df_vr_band_student,df_ethnic_group, df_school_denomination], axis = 1)
    return df_final, df['Exam Score']

df_male, y_male = one_hot_encoding_transformation(df_male)
df_female, y_female = one_hot_encoding_transformation(df_female)
df_mixed, y_mixed = one_hot_encoding_transformation(df_mixed)

# Splitting each dataset (domain) into train - 60%, test - 20% and dev - 20%
def split_dataset(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
    X_train, X_dev, y_train, y_dev = train_test_split(X_train, y_train, test_size=0.25, random_state=1)
    return X_train, y_train, X_test, y_test, X_dev, y_dev

def create_domains(domain):
    if domain == 'male':
        X_male_train, y_male_train, X_male_test, y_male_test, X_male_dev, y_male_dev = split_dataset(df_male, y_male)
        return X_male_train, y_male_train, X_male_test, y_male_test, X_male_dev, y_male_dev
    if domain == 'female':
        X_female_train, y_female_train, X_female_test, y_female_test, X_female_dev, y_female_dev= split_dataset(df_female, y_female)
        return X_female_train, y_female_train, X_female_test, y_female_test, X_female_dev, y_female_dev
    if domain == 'mixed':
        X_mixed_train, y_mixed_train, X_mixed_test, y_mixed_test, X_mixed_dev, y_mixed_dev= split_dataset(df_mixed, y_mixed)
        return X_mixed_train, y_mixed_train, X_mixed_test, y_mixed_test, X_mixed_dev, y_mixed_dev

### Models

In [9]:
def evaluate_on_linear_model(X_train, y_train, X_test, y_test, X_dev, y_dev):
    print('--> Linear Model Lasso')
    X_train = preprocessing.scale(X_train)
    X_test = preprocessing.scale(X_test)
    X_dev = preprocessing.scale(X_dev)
    lasso = linear_model.Lasso()
    parameters = { 'alpha': np.logspace(-4, -0.5, 30)}
    cv = KFold(n_splits = 10, random_state = 42)
    lasso_regressor = GridSearchCV(lasso, parameters, cv = cv)
    lasso_regressor.fit(X_dev, y_dev)    
    y_dev_pred = lasso_regressor.predict(X_dev)
    print('MSE Dev:', mean_squared_error(y_dev, y_dev_pred))
    print("Best: %f using %s" % (lasso_regressor.best_score_, lasso_regressor.best_params_))
    lasso = linear_model.Lasso()
    lasso.set_params = lasso_regressor.best_params_
    lasso.fit(X_train, y_train)
    y_train_pred = lasso.predict(X_train)
    print('MSE Train:', mean_squared_error(y_train, y_train_pred))
    y_test_pred = lasso.predict(X_test)
    return y_test_pred
    
def evaluate_on_neural_network(X_train, y_train, X_test, y_test, X_dev, y_dev):
    print('--> Multi-layer Perceptron regressor')
    X_train = preprocessing.scale(X_train)
    X_test = preprocessing.scale(X_test)
    X_dev = preprocessing.scale(X_dev)
  
    param_grid = {'hidden_layer_sizes': [100, 200],
          'solver': ['adam'],
          'learning_rate': ['invscaling'],
          'learning_rate_init': [1e-1, 1e-2, 1e-3, 1e-4],
          'alpha': [1e-1, 1e-2, 1e-3, 1e-4],
          'epsilon': [1, 1e-1, 1e-2, 1e-3, 1e-4],        
          'early_stopping': [True]}
    
    regr = MLPRegressor(random_state=1, max_iter=500)
    cv = KFold(n_splits = 10, random_state = 42)
    model_grid = GridSearchCV(estimator=regr, param_grid=param_grid, n_jobs=-1,scoring='neg_mean_squared_error', cv= cv)
    grid_result = model_grid.fit(X_dev, y_dev)
    y_dev_pred = grid_result.predict(X_dev)
    print('MSE Dev:', mean_squared_error(y_dev, y_dev_pred))
    print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
    final_model = MLPRegressor(random_state=1, max_iter=500)
    final_model.set_params =  grid_result.best_params_
    final_model.fit(X_train, y_train)
    y_train_pred = final_model.predict(X_train)
    print('MSE Train:', mean_squared_error(y_train, y_train_pred))
    y_test_pred = final_model.predict(X_test)
    return y_test_pred

# Method to concatenate X and y
def add_prediction_features(X, y):
    df_y = pd.DataFrame(y)
    df_y.reset_index(drop=True, inplace=True)  
    X.reset_index(drop=True, inplace=True)  
    return pd.concat([X, df_y], axis = 1)

def get_mse(y_test_pred_lin, y_test_pred_nn, y_test):
    print()
    print('--> Linear Model Lasso')
    print('MSE Test:', mean_squared_error(y_test, y_test_pred_lin))
    print('--> Multi-layer Perceptron regressor')
    print('MSE Test:', mean_squared_error(y_test, y_test_pred_nn))
    print()

### Baseline methods and FEDA

In [18]:
def FEDA(source_1, source_2, target, scale = 1):
    target = np.c_[target, target, np.zeros(target.shape), np.zeros(target.shape)]
    source_1 = np.c_[source_1, np.zeros(source_1.shape), source_1, np.zeros(source_1.shape)]
    source_2 = np.c_[source_2 , np.zeros(source_2.shape), np.zeros(source_2.shape), source_2]
    source = np.r_[source_1, source_2]
    return source, target

## Hyperparameter Sensitivity, features
def check_hyperparameter_behaviour(X_train,y_train, X_test, y_test, X_dev, y_dev):
    print('General')
    X_general_test = np.c_[X_test, np.zeros(X_test.shape), np.zeros(X_test.shape), np.zeros(X_test.shape)]
    get_predictions_from_baseline(X_train,y_train, X_general_test, y_test, X_dev, y_dev)
    print('Target')
    X_target_test = np.c_[X_test, X_test, np.zeros(X_test.shape), np.zeros(X_test.shape)]
    get_predictions_from_baseline(X_train,y_train, X_target_test, y_test, X_dev, y_dev)
    print('Domain 1')
    X_domain_1_test = np.c_[X_test, np.zeros(X_test.shape), X_test, np.zeros(X_test.shape)]
    get_predictions_from_baseline(X_train,y_train, X_domain_1_test, y_test, X_dev, y_dev)
    print('Domain 2')
    X_domain_2_test = np.c_[X_test, np.zeros(X_test.shape), np.zeros(X_test.shape), X_test]
    get_predictions_from_baseline(X_train,y_train, X_domain_2_test, y_test, X_dev, y_dev)

def get_predictions_from_baseline(X_train,y_train,  X_test, y_test, X_dev, y_dev):
    y_pred_lin = evaluate_on_linear_model(X_train, y_train, X_test, y_test, X_dev, y_dev)
    y_pred_nn = evaluate_on_neural_network(X_train, y_train, X_test, y_test, X_dev, y_dev)
    get_mse(y_pred_lin, y_pred_nn, y_test)  
    return y_pred_lin, y_pred_nn
    
def downsampling_for_weight(X_source, y_source, X_target, y_target):
    target_len = len(X_target)
    source = pd.concat([X_source, y_source], axis = 1)    
    downsampled_source = resample(source, n_samples = target_len, random_state = 99)
    downsampled_source_y = downsampled_source['Exam Score']
    downsampled_source_X = downsampled_source.drop(['Exam Score'], axis=1)
    X = pd.concat([downsampled_source_X, X_target]) 
    y = pd.concat([downsampled_source_y, y_target])
    return X, y

def find_best_weight(src_predictions_1, tgt_predictions_1, src_predictions_2, tgt_predictions_2, y_test):
    src_predictions_1 = pd.DataFrame(src_predictions_1)
    tgt_predictions_1 = pd.DataFrame(tgt_predictions_1 )
    src_predictions_2 = pd.DataFrame(src_predictions_2)
    tgt_predictions_2 = pd.DataFrame(tgt_predictions_2)
    
    for i in np.arange(0, 1, 0.1):
        print('Weight: ', i, 1 - i)
        # linear model
        new_src_predictions_1 = src_predictions_1 * i
        new_tgt_predictions_1 = tgt_predictions_1 * (1 - i)
        predictions_1 = new_src_predictions_1 + new_tgt_predictions_1
        # Neural network
        new_src_predictions_2 = src_predictions_2 * i
        new_tgt_predictions_2 = tgt_predictions_2 * (1 - i)
        predictions_2 = new_src_predictions_2 + new_tgt_predictions_2 
        get_mse(predictions_1, predictions_2, y_test)
        
def evaluate_baselines(target, instance_size):
    
    dev_instance_size = 100
    domains = ['male', 'female', 'mixed']
    domains.remove(target)
    source = domains
    X_source_1_train, y_source_1_train, X_source_1_test, y_source_1_test, X_source_1_dev, y_source_1_dev = create_domains(source[0])
    X_source_2_train, y_source_2_train, X_source_2_test, y_source_2_test, X_source_2_dev, y_source_2_dev = create_domains(source[1])

    print('_'*20, 'Scenario 1: SRCONLY','_'*20)  
    print('* SRCONLY Results *')
    X_target_train, y_target_train, X_target_test, y_target_test, X_target_dev, y_target_dev = create_domains(target)
    X_train = pd.concat([X_source_1_train, X_source_2_train])
    y_train = pd.concat([y_source_1_train, y_source_2_train])
    X_dev = X_target_dev[:dev_instance_size]
    y_dev = y_target_dev[:dev_instance_size]
    X_test = X_target_test
    y_test = y_target_test
    y_srconly_pred_lin, y_srconly_pred_nn = get_predictions_from_baseline(X_train,y_train, X_test, y_test, X_dev, y_dev)
    print('* SRCONLY FEDA Results *')
    X_source_train, X_target_train = FEDA(X_source_1_train, X_source_2_train, X_target_train[:instance_size])
    X_source_test, X_target_test = FEDA(X_source_1_test, X_source_2_test, X_target_test)
    X_source_dev, X_target_dev = FEDA(X_source_1_dev, X_source_2_dev, X_target_dev[:dev_instance_size])
    y_srconly_feda_pred_lin, y_srconly_feda_pred_nn = get_predictions_from_baseline(X_source_train,y_train, X_target_test, y_test, X_target_dev, y_dev)
    print('Checking hyperparameters')
    check_hyperparameter_behaviour(X_source_train,y_train, X_test, y_test, X_target_dev, y_dev)

    print('_'*20, 'Scenario 1: TGTONLY','_'*20)
    print('* TGTONLY Results *')
    X_target_train, y_target_train, X_target_test, y_target_test, X_target_dev, y_target_dev = create_domains(target)
    X_train = X_target_train[:instance_size]
    y_train = y_target_train[:instance_size]
    X_dev = X_target_dev[:dev_instance_size]
    y_dev = y_target_dev[:dev_instance_size]
    X_test = X_target_test
    y_test = y_target_test
    y_tgtonly_pred_lin, y_tgtonly_pred_nn = get_predictions_from_baseline(X_train,y_train, X_test, y_test, X_dev, y_dev)
    print('* TGTONLY FEDA Results *')
    X_source_train, X_target_train = FEDA(X_source_1_train, X_source_2_train, X_target_train[:instance_size])
    X_source_test, X_target_test = FEDA(X_source_1_test, X_source_2_test, X_target_test)
    X_source_dev, X_target_dev = FEDA(X_source_1_dev, X_source_2_dev, X_target_dev[:dev_instance_size])
    y_tgtonly_feda_pred_lin, y_tgtonly_feda_pred_nn = get_predictions_from_baseline(X_target_train, y_train, X_target_test, y_test, X_target_dev, y_dev)
    print('Checking hyperparameters')
    check_hyperparameter_behaviour(X_target_train,y_train, X_test, y_test, X_target_dev, y_dev)

    print('_'*20, 'Scenario 1: ALL','_'*20)
    print('* ALL Results *')
    X_target_train, y_target_train, X_target_test, y_target_test, X_target_dev, y_target_dev = create_domains(target)
    X_train = pd.concat([X_source_1_train, X_source_2_train, X_target_train[:instance_size]])
    y_train = pd.concat([y_source_1_train, y_source_2_train, y_target_train[:instance_size]])
    X_dev = X_target_dev[:dev_instance_size]
    y_dev = y_target_dev[:dev_instance_size]
    X_test = X_target_test
    y_test = y_target_test
    get_predictions_from_baseline(X_train,y_train, X_test, y_test, X_dev, y_dev)
    print('* ALL FEDA Results *')
    X_source_train, X_target_train = FEDA(X_source_1_train, X_source_2_train, X_target_train[:instance_size])
    X_source_test, X_target_test = FEDA(X_source_1_test, X_source_2_test, X_target_test)
    X_source_dev, X_target_dev = FEDA(X_source_1_dev, X_source_2_dev, X_target_dev[:dev_instance_size])
    X_train = np.r_[X_source_train, X_target_train]
    get_predictions_from_baseline(X_train, y_train, X_target_test, y_test, X_target_dev, y_dev)
    print('Checking hyperparameters')
    check_hyperparameter_behaviour(X_train,y_train, X_test, y_test, X_target_dev, y_dev)

    print('_'*20, 'Scenario 1: WEIGHTED','_'*20)
    print('* WEIGHTED Results *')
    X_target_train, y_target_train, X_target_test, y_target_test, X_target_dev, y_target_dev = create_domains(target)
    X_target = X_target_train[:instance_size]
    y_target = y_target_train[:instance_size]
    X_1, y_1 = downsampled_X_source_1 = downsampling_for_weight(X_source_1_train, y_source_1_train, X_target, y_target)
    X_2, y_2 = downsampled_X_source_2 = downsampling_for_weight(X_source_2_train, y_source_2_train, X_target, y_target)
    X_train = pd.concat([X_1, X_2, X_target]) 
    y_train = pd.concat([y_1, y_2, y_target])
    X_dev = X_target_dev[:dev_instance_size]
    y_dev = y_target_dev[:dev_instance_size]
    X_test = X_target_test
    y_test = y_target_test
    get_predictions_from_baseline(X_train, y_train, X_test, y_test, X_dev, y_dev)
    print('* WEIGHTED FEDA Results *')
    X_source_train, X_target_train = FEDA(X_1, X_2, X_target_train[:instance_size])
    X_source_test, X_target_test = FEDA(X_source_1_test, X_source_2_test, X_target_test)
    X_source_dev, X_target_dev = FEDA(X_source_1_dev, X_source_2_dev, X_target_dev[:dev_instance_size])
    X_train = np.r_[X_source_train, X_target_train]
    get_predictions_from_baseline(X_train, y_train, X_target_test, y_test, X_target_dev, y_dev)
    print('Checking hyperparameters')
    check_hyperparameter_behaviour(X_train,y_train, X_test, y_test, X_target_dev, y_dev)

    print('_'*20, 'Scenario 1: PRED','_'*20)
    X_target_train, y_target_train, X_target_test, y_target_test, X_target_dev, y_target_dev = create_domains(target)

    # SRCONLY predictions
    # Training data
    X_train = pd.concat([X_source_1_train, X_source_2_train])
    y_train = pd.concat([y_source_1_train, y_source_2_train])
    X_test = X_target_train[:instance_size]
    y_test = y_target_train[:instance_size]
    X_dev = X_target_dev[:dev_instance_size]
    y_dev = y_target_dev[:dev_instance_size]

    predictions_train_1 , predictions_train_2 = get_predictions_from_baseline(X_train, y_train, X_test, y_test, X_dev, y_dev)
    X_train = X_target_train[:instance_size]
    df_train_final_1 = add_prediction_features(X_train, predictions_train_1)
    df_train_final_2 = add_prediction_features(X_train, predictions_train_2)

    # Testing data
    X_train = pd.concat([X_source_1_test, X_source_2_test])
    y_train = pd.concat([y_source_1_test, y_source_2_test])
    X_test = X_target_test
    y_test = y_target_test
    predictions_test_1 , predictions_test_2 = get_predictions_from_baseline(X_train, y_train, X_test, y_test, X_dev, y_dev)
    df_test_final_1 = add_prediction_features(X_target_test, predictions_test_1)
    df_test_final_2 = add_prediction_features(X_target_test, predictions_test_2)

    # Development data
    X_train = pd.concat([X_source_1_dev, X_source_2_dev])
    y_train = pd.concat([y_source_1_dev, y_source_2_dev])
    X_test = X_target_dev[:dev_instance_size]
    y_test = y_target_dev[:dev_instance_size]
    predictions_dev_1 , predictions_dev_2 = get_predictions_from_baseline(X_train, y_train, X_test, y_test, X_dev, y_dev)
    X_target_dev = X_target_dev[:dev_instance_size]
    df_dev_final_1 = add_prediction_features(X_target_dev,predictions_dev_1)
    df_dev_final_2 = add_prediction_features(X_target_dev,predictions_dev_2)

    y_train = y_target_train[:instance_size]
    y_test = y_target_test
    y_dev = y_target_dev[:dev_instance_size]

    print('* PRED Results *')
    y_test_pred_lin = evaluate_on_linear_model(df_train_final_1, y_train, df_test_final_1, y_test, df_dev_final_1, y_dev)
    y_test_pred_nn = evaluate_on_neural_network(df_train_final_2, y_train, df_test_final_2, y_test, df_dev_final_2, y_dev)
    get_mse(y_test_pred_lin, y_test_pred_nn, y_test)

    print('_'*20, 'Scenario 1: LININT','_'*20)
    X_target_train, y_target_train, X_target_test, y_target_test, X_target_dev, y_target_dev = create_domains(target)
    print('* LININT Results *')
    find_best_weight(y_srconly_pred_lin, y_tgtonly_pred_lin, y_srconly_pred_nn, y_tgtonly_pred_nn, y_test)
    print('* LININT FEDA Results *')
    find_best_weight(y_srconly_feda_pred_lin, y_tgtonly_feda_pred_lin, y_srconly_feda_pred_nn, y_tgtonly_feda_pred_nn, y_test)

def run_scenarios():
    n_instances = [100, 600, 1100, 1600, 2100]
    for instances in n_instances:
        print('Number of instances:', instances)
        print('*'*100)
        print('Scenario 1: Male is target domain:')
        print('*'*100)
        evaluate_baselines('male',instances)
        print('*'*100)
        print('Scenario 2: Female is target domain:')
        print('*'*100)
        evaluate_baselines('female',instances)
        print('*'*100)
        print('Scenario 3: Mixed is target domain:')
        print('*'*100)
        evaluate_baselines('mixed', instances)
        
run_scenarios()

Number of instances: 100
****************************************************************************************************
Scenario 1: Male is target domain:
****************************************************************************************************
____________________ Scenario 1: SRCONLY ____________________
* SRCONLY Results *
--> Linear Model Lasso
MSE Dev: 71.06837724894115
Best: 0.204351 using {'alpha': 0.31622776601683794}
MSE Train: 112.587841323962
--> Multi-layer Perceptron regressor
MSE Dev: 86.71825093652642
Best: -115.878823 using {'alpha': 0.0001, 'early_stopping': True, 'hidden_layer_sizes': 200, 'learning_rate': 'invscaling', 'learning_rate_init': 0.01, 'solver': 'adam'}
MSE Train: 93.19097930564791

--> Linear Model Lasso
MSE Test: 122.73136473963345
--> Multi-layer Perceptron regressor
MSE Test: 126.8480412515774

* SRCONLY FEDA Results *
--> Linear Model Lasso
MSE Dev: 71.06801454617023
Best: 0.204356 using {'alpha': 0.31622776601683794}
MSE Train: 110.57

MSE Train: 87.39805361143429

--> Linear Model Lasso
MSE Test: 123.36396902786089
--> Multi-layer Perceptron regressor
MSE Test: 248.5478678596884

Target
--> Linear Model Lasso
MSE Dev: 71.06801454617023
Best: 0.204356 using {'alpha': 0.31622776601683794}
MSE Train: 111.26226328970931
--> Multi-layer Perceptron regressor
MSE Dev: 119.67222406584169
Best: -114.305620 using {'alpha': 0.001, 'early_stopping': True, 'hidden_layer_sizes': 100, 'learning_rate': 'invscaling', 'learning_rate_init': 0.1, 'solver': 'adam'}
MSE Train: 87.39805361143429

--> Linear Model Lasso
MSE Test: 123.36396902786089
--> Multi-layer Perceptron regressor
MSE Test: 231.8834165188848

Domain 1
--> Linear Model Lasso
MSE Dev: 71.06801454617023
Best: 0.204356 using {'alpha': 0.31622776601683794}
MSE Train: 111.26226328970931
--> Multi-layer Perceptron regressor
MSE Dev: 119.67222406584169
Best: -114.305620 using {'alpha': 0.001, 'early_stopping': True, 'hidden_layer_sizes': 100, 'learning_rate': 'invscaling', 'le

--> Linear Model Lasso
MSE Test: 122.31955904150063
--> Multi-layer Perceptron regressor
MSE Test: 220.27904286968658

****************************************************************************************************
Scenario 2: Female is target domain:
****************************************************************************************************
____________________ Scenario 1: SRCONLY ____________________
* SRCONLY Results *
--> Linear Model Lasso
MSE Dev: 109.36817042031873
Best: 0.234644 using {'alpha': 0.31622776601683794}
MSE Train: 115.8330989982791
--> Multi-layer Perceptron regressor
MSE Dev: 93.48562968262281
Best: -177.607549 using {'alpha': 0.1, 'early_stopping': True, 'hidden_layer_sizes': 200, 'learning_rate': 'invscaling', 'learning_rate_init': 0.001, 'solver': 'adam'}
MSE Train: 96.57359923745553

--> Linear Model Lasso
MSE Test: 155.88956915874624
--> Multi-layer Perceptron regressor
MSE Test: 152.50434075622093

* SRCONLY FEDA Results *
--> Linear Model Lasso

MSE Train: 91.20114302881059

--> Linear Model Lasso
MSE Test: 155.1856298736129
--> Multi-layer Perceptron regressor
MSE Test: 362.2379014634781

Target
--> Linear Model Lasso
MSE Dev: 109.36810817340125
Best: 0.234641 using {'alpha': 0.31622776601683794}
MSE Train: 115.81807973601535
--> Multi-layer Perceptron regressor
MSE Dev: 83.18531873397544
Best: -180.899357 using {'alpha': 0.1, 'early_stopping': True, 'hidden_layer_sizes': 100, 'learning_rate': 'invscaling', 'learning_rate_init': 0.1, 'solver': 'adam'}
MSE Train: 91.20114302881059

--> Linear Model Lasso
MSE Test: 155.1856298736129
--> Multi-layer Perceptron regressor
MSE Test: 337.82039544572433

Domain 1
--> Linear Model Lasso
MSE Dev: 109.36810817340125
Best: 0.234641 using {'alpha': 0.31622776601683794}
MSE Train: 115.81807973601535
--> Multi-layer Perceptron regressor
MSE Dev: 83.18531873397544
Best: -180.899357 using {'alpha': 0.1, 'early_stopping': True, 'hidden_layer_sizes': 100, 'learning_rate': 'invscaling', 'learnin

MSE Test: 143.40622970668244
--> Multi-layer Perceptron regressor
MSE Test: 254.75606560277964

Weight:  0.8 0.19999999999999996

--> Linear Model Lasso
MSE Test: 147.09957663144394
--> Multi-layer Perceptron regressor
MSE Test: 286.49424441447525

Weight:  0.9 0.09999999999999998

--> Linear Model Lasso
MSE Test: 151.26068978213198
--> Multi-layer Perceptron regressor
MSE Test: 322.7093669733819

****************************************************************************************************
Scenario 3: Mixed is target domain:
****************************************************************************************************
____________________ Scenario 1: SRCONLY ____________________
* SRCONLY Results *
--> Linear Model Lasso
MSE Dev: 86.24903100732917
Best: 0.157143 using {'alpha': 0.31622776601683794}
MSE Train: 121.42839905708067
--> Multi-layer Perceptron regressor
MSE Dev: 106.89924675960377
Best: -126.539141 using {'alpha': 0.1, 'early_stopping': True, 'hidden_layer_sizes

...

____________________ Scenario 1: SRCONLY ____________________
* SRCONLY Results *
--> Linear Model Lasso
MSE Dev: 109.36817042031873
Best: 0.234644 using {'alpha': 0.31622776601683794}
MSE Train: 115.8330989982791
--> Multi-layer Perceptron regressor
MSE Dev: 93.48562968262281
Best: -177.607549 using {'alpha': 0.1, 'early_stopping': True, 'hidden_layer_sizes': 200, 'learning_rate': 'invscaling', 'learning_rate_init': 0.001, 'solver': 'adam'}
MSE Train: 96.57359923745553

--> Linear Model Lasso
MSE Test: 155.88956915874624
--> Multi-layer Perceptron regressor
MSE Test: 152.50434075622093

* SRCONLY FEDA Results *
--> Linear Model Lasso
MSE Dev: 109.36810817340125
Best: 0.234641 using {'alpha': 0.31622776601683794}
MSE Train: 115.83309899827903
--> Multi-layer Perceptron regressor
MSE Dev: 83.18531873397544
Best: -180.899357 using {'alpha': 0.1, 'early_stopping': True, 'hidden_layer_sizes': 100, 'learning_rate': 'invscaling', 'learning_rate_init': 0.1, 'solver': 'adam'}
MSE Train: 92.326

MSE Dev: 83.18531873397544
Best: -180.899357 using {'alpha': 0.1, 'early_stopping': True, 'hidden_layer_sizes': 100, 'learning_rate': 'invscaling', 'learning_rate_init': 0.1, 'solver': 'adam'}
MSE Train: 89.5430821039526

--> Linear Model Lasso
MSE Test: 144.9399595900929
--> Multi-layer Perceptron regressor
MSE Test: 295.3546498600191

Domain 1
--> Linear Model Lasso
MSE Dev: 109.36810817340125
Best: 0.234641 using {'alpha': 0.31622776601683794}
MSE Train: 115.48432037617812
--> Multi-layer Perceptron regressor
MSE Dev: 83.18531873397544
Best: -180.899357 using {'alpha': 0.1, 'early_stopping': True, 'hidden_layer_sizes': 100, 'learning_rate': 'invscaling', 'learning_rate_init': 0.1, 'solver': 'adam'}
MSE Train: 89.5430821039526

--> Linear Model Lasso
MSE Test: 146.7028555489817
--> Multi-layer Perceptron regressor
MSE Test: 361.10027264911866

Domain 2
--> Linear Model Lasso
MSE Dev: 109.36810817340125
Best: 0.234641 using {'alpha': 0.31622776601683794}
MSE Train: 115.48432037617812


MSE Dev: 86.24903100732917
Best: 0.157143 using {'alpha': 0.31622776601683794}
MSE Train: 121.42839905708067
--> Multi-layer Perceptron regressor
MSE Dev: 106.89924675960377
Best: -126.539141 using {'alpha': 0.1, 'early_stopping': True, 'hidden_layer_sizes': 200, 'learning_rate': 'invscaling', 'learning_rate_init': 0.1, 'solver': 'adam'}
MSE Train: 97.10631555413285

--> Linear Model Lasso
MSE Test: 106.17271774133052
--> Multi-layer Perceptron regressor
MSE Test: 109.3879607615141

* SRCONLY FEDA Results *
--> Linear Model Lasso
MSE Dev: 86.24902408451032
Best: 0.157140 using {'alpha': 0.31622776601683794}
MSE Train: 118.00322917595912
--> Multi-layer Perceptron regressor
MSE Dev: 136.26758715252544
Best: -124.900134 using {'alpha': 0.1, 'early_stopping': True, 'hidden_layer_sizes': 100, 'learning_rate': 'invscaling', 'learning_rate_init': 0.1, 'solver': 'adam'}
MSE Train: 91.52290676649376

--> Linear Model Lasso
MSE Test: 107.85144469608962
--> Multi-layer Perceptron regressor
MSE T

MSE Dev: 136.26758715252544
Best: -124.900134 using {'alpha': 0.1, 'early_stopping': True, 'hidden_layer_sizes': 100, 'learning_rate': 'invscaling', 'learning_rate_init': 0.1, 'solver': 'adam'}
MSE Train: 88.08376243398745

--> Linear Model Lasso
MSE Test: 105.77093374466891
--> Multi-layer Perceptron regressor
MSE Test: 224.94534735411483

Domain 1
--> Linear Model Lasso
MSE Dev: 86.24902408451032
Best: 0.157140 using {'alpha': 0.31622776601683794}
MSE Train: 116.03245356972454
--> Multi-layer Perceptron regressor
MSE Dev: 136.26758715252544
Best: -124.900134 using {'alpha': 0.1, 'early_stopping': True, 'hidden_layer_sizes': 100, 'learning_rate': 'invscaling', 'learning_rate_init': 0.1, 'solver': 'adam'}
MSE Train: 88.08376243398745

--> Linear Model Lasso
MSE Test: 105.77093374466891
--> Multi-layer Perceptron regressor
MSE Test: 197.6718462274462

Domain 2
--> Linear Model Lasso
MSE Dev: 86.24902408451032
Best: 0.157140 using {'alpha': 0.31622776601683794}
MSE Train: 116.03245356972

Weight:  0.9 0.09999999999999998

--> Linear Model Lasso
MSE Test: 107.09601968547248
--> Multi-layer Perceptron regressor
MSE Test: 203.84662701089567



### Return of Frustratingly Easy Domain Adaptation: CORAL

In [19]:
def algorithm_CORAL(source, target):
    cov_source_train = np.cov(source.T) + np.identity(source.shape[1])
    cov_target_train = np.cov(target.T) + np.identity(target.shape[1])
    source = np.dot(source, sc_linalg.fractional_matrix_power(cov_source_train, -0.5))
    X_source = np.dot(source, sc_linalg.fractional_matrix_power(cov_target_train, 0.5))
    return X_source
    
def CORAL(target, instance_size):
    dev_instance_size = 100
    domains = ['male', 'female', 'mixed']
    domains.remove(target)
    source = domains
    X_source_1_train, y_source_1_train, X_source_1_test, y_source_1_test, X_source_1_dev, y_source_1_dev = create_domains(source[0])
    X_source_2_train, y_source_2_train, X_source_2_test, y_source_2_test, X_source_2_dev, y_source_2_dev = create_domains(source[1])
    X_target_train, y_target_train, X_target_test, y_target_test, X_target_dev, y_target_dev = create_domains(target)

    print('* CORAL SRCONLY Results *')
    source_train = pd.concat([X_source_1_train, X_source_2_train])
    target_train = X_target_train[:instance_size]
    X_source = algorithm_CORAL(source_train, target_train)
    y_source = pd.concat([y_source_1_train, y_source_2_train])
    X_test = X_target_test
    y_test = y_target_test
    X_dev = X_target_dev[:dev_instance_size]
    y_dev = y_target_dev[:dev_instance_size]
    src_predictions_1, src_predictions_2 = get_predictions_from_baseline(X_source, y_source, X_test, y_test, X_dev, y_dev)
    
    print('* CORAL TGTONLY Results *')
    X_target_train, y_target_train, X_target_test, y_target_test, X_target_dev, y_target_dev = create_domains(target)
    source_train = X_target_train[:instance_size]
    target_train = X_target_train[:instance_size]
    X_source = algorithm_CORAL(source_train, target_train)
    y_source = y_target_train[:instance_size]
    X_test = X_target_test
    y_test = y_target_test
    X_dev = X_target_dev[:dev_instance_size]
    y_dev = y_target_dev[:dev_instance_size]
    tgt_predictions_1, tgt_predictions_2 = get_predictions_from_baseline(X_source, y_source, X_test, y_test, X_dev, y_dev)
    
    print('* CORAL ALL Results *')
    X_target_train, y_target_train, X_target_test, y_target_test, X_target_dev, y_target_dev = create_domains(target)
    target_train = X_target_train[:instance_size]
    y_target_train = y_target_train[:instance_size]
    source_train = pd.concat([X_source_1_train, X_source_2_train, target_train])
    X_source = algorithm_CORAL(source_train, target_train)
    y_source = pd.concat([y_source_1_train, y_source_2_train, y_target_train])
    X_test = X_target_test
    y_test = y_target_test
    X_dev = X_target_dev[:dev_instance_size]
    y_dev = y_target_dev[:dev_instance_size]
    get_predictions_from_baseline(X_source, y_source, X_test, y_test, X_dev, y_dev)
    
    print('* CORAL WEIGHT Results *')
    X_target_train, y_target_train, X_target_test, y_target_test, X_target_dev, y_target_dev = create_domains(target)
    target_train = X_target_train[:instance_size]
    y_target_train = y_target_train[:instance_size]
    source_train = pd.concat([X_source_1_train, X_source_2_train])
    y_source = pd.concat([y_source_1_train, y_source_2_train])
    downsample_source_train, y_train = downsampling_for_weight(source_train, y_source, target_train, y_target_train)
    X_source = algorithm_CORAL(downsample_source_train, target_train)
    y_target_train = y_target_train[:instance_size]
    y_source = y_train
    X_test = X_target_test
    y_test = y_target_test
    X_dev = X_target_dev[:dev_instance_size]
    y_dev = y_target_dev[:dev_instance_size]
    get_predictions_from_baseline(X_source, y_source, X_test, y_test, X_dev, y_dev)
    
    print('* CORAL LININT Results *')
    find_best_weight(src_predictions_1, tgt_predictions_1, src_predictions_2, tgt_predictions_2, y_test)
    
def run_CORAL_with_n_instances():
    print('*'*100)
    print('CORAL')
    print('*'*100)
    domains = ['male', 'female', 'mixed']
    n_instances = [100, 300, 500, 700]
    
    for domain in domains:
        print('Domain:', domain)
        print()
        for instances in n_instances:
            print('Number of instances:', instances)
            CORAL(domain, instances) 
        print('*'*100)

run_CORAL_with_n_instances()

****************************************************************************************************
CORAL
****************************************************************************************************
Domain: male

Number of instances: 100
* CORAL SRCONLY Results *
--> Linear Model Lasso
MSE Dev: 71.06837724894115
Best: 0.204351 using {'alpha': 0.31622776601683794}
MSE Train: 112.89954432324721
--> Multi-layer Perceptron regressor
MSE Dev: 86.71825093652642
Best: -115.878823 using {'alpha': 0.0001, 'early_stopping': True, 'hidden_layer_sizes': 200, 'learning_rate': 'invscaling', 'learning_rate_init': 0.01, 'solver': 'adam'}
MSE Train: 92.82433443531427

--> Linear Model Lasso
MSE Test: 123.59874317030716
--> Multi-layer Perceptron regressor
MSE Test: 130.56011004990154

* CORAL TGTONLY Results *
--> Linear Model Lasso
MSE Dev: 71.06837724894115
Best: 0.204351 using {'alpha': 0.31622776601683794}
MSE Train: 118.1367248438925
--> Multi-layer Perceptron regressor
MSE Dev: 86.718250

MSE Dev: 71.06837724894115
Best: 0.204351 using {'alpha': 0.31622776601683794}
MSE Train: 119.63566553553737
--> Multi-layer Perceptron regressor
MSE Dev: 86.71825093652642
Best: -115.878823 using {'alpha': 0.0001, 'early_stopping': True, 'hidden_layer_sizes': 200, 'learning_rate': 'invscaling', 'learning_rate_init': 0.01, 'solver': 'adam'}
MSE Train: 87.70068132291333

--> Linear Model Lasso
MSE Test: 121.12469902322103
--> Multi-layer Perceptron regressor
MSE Test: 130.75658328491136

* CORAL ALL Results *
--> Linear Model Lasso
MSE Dev: 71.06837724894115
Best: 0.204351 using {'alpha': 0.31622776601683794}
MSE Train: 113.76895948562313
--> Multi-layer Perceptron regressor
MSE Dev: 86.71825093652642
Best: -115.878823 using {'alpha': 0.0001, 'early_stopping': True, 'hidden_layer_sizes': 200, 'learning_rate': 'invscaling', 'learning_rate_init': 0.01, 'solver': 'adam'}
MSE Train: 94.34248670664607

--> Linear Model Lasso
MSE Test: 122.77840821885493
--> Multi-layer Perceptron regressor
M

MSE Train: 96.87683753165479

--> Linear Model Lasso
MSE Test: 155.34493310664914
--> Multi-layer Perceptron regressor
MSE Test: 147.25205232585492

* CORAL WEIGHT Results *
--> Linear Model Lasso
MSE Dev: 109.36817042031873
Best: 0.234644 using {'alpha': 0.31622776601683794}
MSE Train: 90.23933134720272
--> Multi-layer Perceptron regressor
MSE Dev: 93.48562968262281
Best: -177.607549 using {'alpha': 0.1, 'early_stopping': True, 'hidden_layer_sizes': 200, 'learning_rate': 'invscaling', 'learning_rate_init': 0.001, 'solver': 'adam'}
MSE Train: 62.71570506936003

--> Linear Model Lasso
MSE Test: 139.17739970914755
--> Multi-layer Perceptron regressor
MSE Test: 153.55459071079548

* CORAL LININT Results *
Weight:  0.0 1.0

--> Linear Model Lasso
MSE Test: 130.65038633086905
--> Multi-layer Perceptron regressor
MSE Test: 153.87574446239665

Weight:  0.1 0.9

--> Linear Model Lasso
MSE Test: 131.06976560322101
--> Multi-layer Perceptron regressor
MSE Test: 148.28521808115167

Weight:  0.2 0

MSE Dev: 109.36817042031873
Best: 0.234644 using {'alpha': 0.31622776601683794}
MSE Train: 116.02214805500313
--> Multi-layer Perceptron regressor
MSE Dev: 93.48562968262281
Best: -177.607549 using {'alpha': 0.1, 'early_stopping': True, 'hidden_layer_sizes': 200, 'learning_rate': 'invscaling', 'learning_rate_init': 0.001, 'solver': 'adam'}
MSE Train: 96.57698235121475

--> Linear Model Lasso
MSE Test: 155.65565297126798
--> Multi-layer Perceptron regressor
MSE Test: 150.61417911693667

* CORAL TGTONLY Results *
--> Linear Model Lasso
MSE Dev: 109.36817042031873
Best: 0.234644 using {'alpha': 0.31622776601683794}
MSE Train: 100.97059713313894
--> Multi-layer Perceptron regressor
MSE Dev: 93.48562968262281
Best: -177.607549 using {'alpha': 0.1, 'early_stopping': True, 'hidden_layer_sizes': 200, 'learning_rate': 'invscaling', 'learning_rate_init': 0.001, 'solver': 'adam'}
MSE Train: 72.55158626952057

--> Linear Model Lasso
MSE Test: 127.34529733841792
--> Multi-layer Perceptron regressor

MSE Train: 80.1375544448323

--> Linear Model Lasso
MSE Test: 105.64104064686177
--> Multi-layer Perceptron regressor
MSE Test: 108.79151445042278

* CORAL ALL Results *
--> Linear Model Lasso
MSE Dev: 86.24903100732917
Best: 0.157143 using {'alpha': 0.31622776601683794}
MSE Train: 120.95509495513242
--> Multi-layer Perceptron regressor
MSE Dev: 106.89924675960377
Best: -126.539141 using {'alpha': 0.1, 'early_stopping': True, 'hidden_layer_sizes': 200, 'learning_rate': 'invscaling', 'learning_rate_init': 0.1, 'solver': 'adam'}
MSE Train: 97.71788914397037

--> Linear Model Lasso
MSE Test: 105.21070531716825
--> Multi-layer Perceptron regressor
MSE Test: 110.06162354047883

* CORAL WEIGHT Results *
--> Linear Model Lasso
MSE Dev: 86.24903100732917
Best: 0.157143 using {'alpha': 0.31622776601683794}
MSE Train: 115.26359060019391
--> Multi-layer Perceptron regressor
MSE Dev: 106.89924675960377
Best: -126.539141 using {'alpha': 0.1, 'early_stopping': True, 'hidden_layer_sizes': 200, 'learn

MSE Dev: 106.89924675960377
Best: -126.539141 using {'alpha': 0.1, 'early_stopping': True, 'hidden_layer_sizes': 200, 'learning_rate': 'invscaling', 'learning_rate_init': 0.1, 'solver': 'adam'}
MSE Train: 92.13032110291564

--> Linear Model Lasso
MSE Test: 103.37070381600977
--> Multi-layer Perceptron regressor
MSE Test: 106.92151072385693

* CORAL LININT Results *
Weight:  0.0 1.0

--> Linear Model Lasso
MSE Test: 103.10762063861826
--> Multi-layer Perceptron regressor
MSE Test: 104.73806674939583

Weight:  0.1 0.9

--> Linear Model Lasso
MSE Test: 102.91522669121116
--> Multi-layer Perceptron regressor
MSE Test: 103.07851196832247

Weight:  0.2 0.8

--> Linear Model Lasso
MSE Test: 102.82846040484576
--> Multi-layer Perceptron regressor
MSE Test: 101.88384839139077

Weight:  0.30000000000000004 0.7

--> Linear Model Lasso
MSE Test: 102.8473217795221
--> Multi-layer Perceptron regressor
MSE Test: 101.15407601860076

Weight:  0.4 0.6

--> Linear Model Lasso
MSE Test: 102.97181081524018